In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_3260214/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp = df_tmp[2000:3000]
df_tmp 

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
10047,LAURA RODING 9335,LA FLORIDA,None,,
10048,PASAJE 17 MZ34 SITIO4 PADRE HURTADO PADRE HURTADO,PADRE HURTADO,None,,
10052,EL CONSUELO SN CEMENTERIO PTE 337176 SALAMANCA...,SALAMANCA,None,,
10056,"PAINE, SANTA TERESA 41 PAINE",BUIN,None,,
10059,VALDIVIA PASAJE,VALDIVIA,None,,
...,...,...,...,...,...
14996,"PA CANAAN 2239, CH ESTRELLA DEL NORTE, ALTO HO...",ALTO HOSPICIO,None,,
15007,"LOS GLADIOLOS 12, VILLA MERCEDES VILLA MERCEDES",TUCAPEL,None,,
15009,LOS PELLINES SN CURIÑANCO CURIÑANCO,VALDIVIA,None,,
15017,BOLIVIA 1972 AVENIDA SANTA ROSA,SAN RAMON,None,,


In [6]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

ddata = dd.from_pandas(df_tmp, npartitions=30)

# Sample meta dataframe to indicate the structure of the output
meta = pd.DataFrame({
    "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
    "COMUNA DOMICILIO": pd.Series(dtype='str'),
    "NUEVA DIRECCION": pd.Series(dtype='str'),
    "X": pd.Series(dtype='float'),
    "Y": pd.Series(dtype='float')
})

def apply_validate_address(df_chunk):
    # Handle missing values before passing to validate_address
    df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
    
    # Apply the validation function to each row and unpack the results
    df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
        lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
        axis=1
    )
    return df_chunk

# Map the function to each partition, with the meta keyword to define the structure
res = ddata.map_partitions(apply_validate_address, meta=meta).compute()

sin coincidencias ['PASAJE 3', 'COPIAPO']  ------  PASAJE 3 COPIAPO
['STA', 'MARTA', '4245', 'LOS AROMOS', 'POBLACION', 'SANTA', 'TERESA', 'ALTO', 'HOSPICIO']  ------  STA MARTA 4245 LOS AROMOS, POBLACION SANTA TERESA ALTO HOSPICIO  -------  Santa Marta, Avenida Juanita, Población Volcán San José 1, Puente Alto, Provincia de Cordillera, Región Metropolitana de Santiago, 8167450, Chile
['CAMINO', 'NERCON', 'ALTO', 'IGLESIA', 'CASTRO']  ------  CAMINO NERCON ALTO IGLESIA NERCON CASTRO  -------  Ruta W-502, Nercón, Castro, Provincia de Chiloé, Región de Los Lagos, 5700808, Chile
['PLACER', '1156', 'DEPTO', '409', 'SANTIAGO', 'CHILE', 'APARTAMENTO']  ------  PLACER 1156, DEPTO 409, SANTIAGO, CHILE APARTAMENTO 409 SANTIAGO  -------  Edificio Bosques de Santiago, 1156, Placer, Barrio Huemul, Santiago, Provincia de Santiago, Región Metropolitana de Santiago, 8360892, Chile
['FREI', 'BONN', 'ORIENTE', '3436', 'POBLACION', 'RENE', 'SCHNNEIDER', 'CALAMA']  ------  FREI BONN ORIENTE 3436 POBLACIO

In [7]:
res

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
10047,LAURA RODING 9335,LA FLORIDA,None,NaN,NaN
10048,PASAJE 17 MZ34 SITIO4 PADRE HURTADO PADRE HURTADO,PADRE HURTADO,"Pasaje 17, Las Aralias, Padre Hurtado, Provinc...",-33.586287,-70.812841
10052,EL CONSUELO SN CEMENTERIO PTE 337176 SALAMANCA...,SALAMANCA,"Ruta N-809, Zapallar - Pedregal, El Carmen, Pr...",-36.850367,-71.804526
10056,"PAINE, SANTA TERESA 41 PAINE",BUIN,"41, Ruta G-500, El Bosque, Valdivia de Paine, ...",-33.803254,-70.853527
10059,VALDIVIA PASAJE,VALDIVIA,None,NaN,NaN
...,...,...,...,...,...
14996,"PA CANAAN 2239, CH ESTRELLA DEL NORTE, ALTO HO...",ALTO HOSPICIO,"Pasaje Canaán, Alto Hospicio, Provincia de Iqu...",-20.245008,-70.094455
15007,"LOS GLADIOLOS 12, VILLA MERCEDES VILLA MERCEDES",TUCAPEL,"Ruta Q-969, Reñico, Tucapel, Provincia de Bío-...",-37.212955,-71.933609
15009,LOS PELLINES SN CURIÑANCO CURIÑANCO,VALDIVIA,None,NaN,NaN
15017,BOLIVIA 1972 AVENIDA SANTA ROSA,SAN RAMON,"Santa Rosa, Autopista Vespucio Sur, Población ...",-33.542388,-70.634127


In [8]:
print("A")

A


In [9]:
res.to_csv('./data/out/ad3.csv')